In [1]:
# Load libraries
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [2]:
# Train data
X_train = pd.read_csv('x_train_smote_oversample.csv')
y_train = pd.read_csv('y_train_smote_oversample.csv')

# Test data
X_test = pd.read_csv('x_test_smote_oversample.csv')
y_test = pd.read_csv('y_test_smote_oversample.csv')

# Validation data
X_validation = pd.read_csv('x_validation_smote_oversample.csv')
y_validation = pd.read_csv('y_validation_smote_oversample.csv')

In [3]:
y_train

,Unnamed: 0,LABEL
0,2195487,0
1,4672398,0
2,6145219,0
3,9506515,0
4,6031975,0
...,...,...
805713,8699843,2
805714,11442832,3
805715,7477652,2
805716,5199567,0


In [4]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train["LABEL"])

#Predict the response for test dataset
y_pred = model.predict(X_test)


In [5]:
y_pred
y_test

,Unnamed: 0,LABEL
0,11136211,3
1,5416532,0
2,443488,0
3,1403896,1
4,8928981,2
...,...,...
441675,11253423,3
441676,2713178,1
441677,4067969,0
441678,8945610,2


In [6]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test["LABEL"], y_pred))

Accuracy: 0.6581393769244702


In [11]:
print('Matriz de confusion', metrics.confusion_matrix(y_test["LABEL"], y_pred))
print(metrics.classification_report(y_test["LABEL"], y_pred, target_names=['0', '1', '2', '3']))

Matriz de confusion [[200046  37944      0      0]
 [     5  90641      0      0]
 [ 81782      0      0      0]
 [ 31262      0      0      0]]


C:\Users\diana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\diana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.64      0.84      0.73    237990
           1       0.70      1.00      0.83     90646
           2       0.00      0.00      0.00     81782
           3       0.00      0.00      0.00     31262

    accuracy                           0.66    441680
   macro avg       0.34      0.46      0.39    441680
weighted avg       0.49      0.66      0.56    441680



C:\Users\diana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
X_test

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0.875925,0.502721,0.989089,0.989052,2.056728e-06,0.001221,0.455390,6.342370e-07,0.0,0.989700,0.426642,0.989700,0.426642,0.157366,0.021535,0.987180
1,1,0.213710,0.000533,0.969503,0.969503,6.896326e-08,0.000809,0.877394,5.273296e-07,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.918759
2,2,0.051345,0.000017,0.001289,0.001289,4.320345e-07,0.006760,0.713024,1.249255e-07,0.0,0.022431,0.001801,0.022431,0.001801,0.000000,0.000000,0.005307
3,3,0.162539,0.000000,0.004009,0.004009,3.245330e-08,0.692134,0.754864,4.804825e-08,0.0,0.015625,0.000000,0.015625,0.000000,0.162946,0.000000,0.016796
4,4,0.620376,0.918591,0.980858,0.980790,2.034416e-06,0.001221,0.920592,7.495528e-07,0.0,0.007736,0.001740,0.007736,0.001740,0.000000,0.000000,0.960777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441675,441675,0.889496,0.501871,0.989610,0.989573,1.973566e-06,0.001221,0.668925,6.342370e-07,0.0,0.989700,0.426642,0.989700,0.426642,0.157366,0.021535,0.988583
441676,441676,0.314126,0.000008,0.007204,0.007204,3.245330e-08,0.374151,0.758770,4.804825e-08,0.0,0.015625,0.000000,0.015625,0.000000,0.162946,0.000000,0.032458
441677,441677,0.057576,0.000133,0.955036,0.955037,1.744365e-07,0.000809,0.885512,2.906919e-07,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.902627
441678,441678,0.622301,0.756190,0.980871,0.980815,2.107436e-06,0.001221,0.155062,6.870900e-07,0.0,0.007736,0.001740,0.007736,0.001740,0.000000,0.000000,0.960976


In [14]:
metrics.plot_roc_curve(abc, X_test, y_test["LABEL"])
plt.show()

C:\Users\diana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function plot_roc_curve is deprecated; Function :func:`plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: :meth:`sklearn.metric.RocCurveDisplay.from_predictions` or :meth:`sklearn.metric.RocCurveDisplay.from_estimator`.
  warnings.warn(msg, category=FutureWarning)


ValueError: Expected 'estimator' to be a binary classifier, but got AdaBoostClassifier fit on multiclass (4 classes) data